# T81-558: Applications of Deep Neural Networks
**Module 7: Generative Adversarial Networks**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module Video Material

Main video lecture:

* [Part 7.1: Introduction to GANS for Image and Data Generation](https://www.youtube.com/watch?v=u8xn393mDPM&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=21)
* [Part 7.2: Implementing a GAN in Keras](https://www.youtube.com/watch?v=cf6FDLFNWEk&index=22&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)
* [Part 7.3: Face Generation with StyleGAN and Python](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)
* [Part 7.4: GANS for Semi-Supervised Learning in Keras](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)
* [Part 7.5: An Overview of GAN Research](https://www.youtube.com/watch?v=LSSH_NdXwhU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=23)


# Part 7.1: Introduction to GANS for Image and Data Generation

# Part 7.2: Implementing a GAN in Keras

# Part 7.3: Face Generation with StyleGAN and Python

# Part 7.4: GANS for Semi-Supervised Learning in Keras

# Part 7.5: An Overview of GAN Research

# Module 7 Assignment

You can find the first assignment here: [assignment 7](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb)